In [1]:
# ─── Cell 1: full mapping + ΔG′° calculation ─────────────────────────────────────
import sys, csv, time, requests
import cobra
from equilibrator_api import ComponentContribution, Q_

# ——— CONFIG ——————————————————————————————————————————————
COBRA_MODEL_FILE = '/Users/benedikthaarscheidt/M.Sc./master_thesis/Models/generic_models/E_coli/iML1515.xml'
BIGG_BASE        = 'http://bigg.ucsd.edu/api/v2'
MODEL_ID         = 'iML1515'
OUTPUT_CSV       = 'reaction_dG0.csv'
PH               = 7.0
IONIC_STRENGTH   = 0.25
# ——————————————————————————————————————————————————————————————

def fetch_bigg_json(path, identifier):
    """Generic GET from BiGG API, return JSON or {}."""
    url = f"{BIGG_BASE}{path}/{identifier}"
    try:
        r = requests.get(url, timeout=5)
        if r.status_code == 200:
            return r.json()
    except:
        pass
    return {}

def pick_equilibrator_id(dblinks):
    """Pick in order KEGG > ChEBI > SEED > MetaNetX from BiGG database_links."""
    for key in ('KEGG Compound','CHEBI','SEED Compound','MetaNetX (MNX) Chemical'):
        if key in dblinks and dblinks[key]:
            return dblinks[key][0]['id']
    return None

# 1) load SBML model
try:
    model = cobra.io.read_sbml_model(COBRA_MODEL_FILE)
except Exception as e:
    print("ERROR loading SBML:", e, file=sys.stderr)
    sys.exit(1)

# 2) metabolite→eQuilibrator ID mapping
mets       = list(model.metabolites)
total      = len(mets)
met2eq     = {}
counts     = dict(kegg=0, chebi=0, seed=0, metanetx=0, fallback=0)
unmapped   = []

print(f"\nMapping {total} metabolites…")
for idx, met in enumerate(mets, 1):
    eq = None

    # 2a) primary BiGG model API
    js = fetch_bigg_json(f"/models/{MODEL_ID}/metabolites", met.id)
    dbl = js.get('database_links', {})
    eq = pick_equilibrator_id(dbl)

    # 2b) fallback to universal if missing
    if not eq:
        base = met.id.split('_')[0]
        js2 = fetch_bigg_json("/universal/metabolites", base)
        dbl2 = js2.get('database_links', {})
        eq = pick_equilibrator_id(dbl2)
        source = 'fallback'
    else:
        source = 'primary'

    if eq:
        met2eq[met.id] = eq
        lid = eq.lower()
        if source == 'fallback':
            counts['fallback'] += 1
        elif lid.startswith('c') and lid[1].isdigit():
            counts['kegg'] += 1
        elif lid.startswith('chebi'):
            counts['chebi'] += 1
        elif lid.startswith('cpd'):
            counts['seed'] += 1
        elif lid.startswith('mnx'):
            counts['metanetx'] += 1
        status = f"{eq} ({source})"
    else:
        unmapped.append(met.id)
        status = "UNMAPPED"

    print(f"[{idx:4d}/{total}] {met.id:12s} → {status}")
    time.sleep(0.005)  # throttle

# 3) report
print("\nMapping summary:")
print(f"  KEGG:     {counts['kegg']:4d}/{total}")
print(f"  ChEBI:    {counts['chebi']:4d}/{total}")
print(f"  SEED:     {counts['seed']:4d}/{total}")
print(f"  MetaNetX: {counts['metanetx']:4d}/{total}")
print(f"  fallback: {counts['fallback']:4d}/{total}")
print(f"  unmapped: {len(unmapped):4d}/{total}")
if unmapped:
    print("  first few unmapped:", unmapped[:10])

MAPPING_CSV = 'metabolite_mapping.csv'
with open(MAPPING_CSV, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['metabolite_id', 'equilibrator_id'])
    for met in mets:
        writer.writerow([met.id, met2eq.get(met.id, '')])
print(f"✔ Saved full metabolite → eQuilibrator mapping to {MAPPING_CSV}")




Mapping 1877 metabolites…
[   1/1877] octapb_c     → MNXM147531 (primary)
[   2/1877] cysi__L_e    → C00491 (primary)
[   3/1877] dhap_c       → C00111 (primary)
[   4/1877] prbatp_c     → C02739 (primary)
[   5/1877] 10fthf_c     → C00234 (primary)
[   6/1877] btal_c       → C01412 (primary)
[   7/1877] 6pgg_c       → MNXM147389 (primary)
[   8/1877] co2_e        → C00011 (primary)
[   9/1877] akg_e        → C00026 (primary)
[  10/1877] gsn_e        → C00387 (primary)
[  11/1877] pydx5p_c     → C00018 (primary)
[  12/1877] 3dhgulnp_c   → C14899 (primary)
[  13/1877] g3ps_c       → CHEBI:61931 (primary)
[  14/1877] adphep_LD_c  → C06398 (primary)
[  15/1877] lyx__L_c     → C01508 (primary)
[  16/1877] din_p        → C05512 (primary)
[  17/1877] 2pg_c        → C00631 (primary)
[  18/1877] ptrc_p       → C00134 (primary)
[  19/1877] malt_p       → C00208 (primary)
[  20/1877] pppn_p       → C05629 (primary)
[  21/1877] arbtn_p      → cpd15411 (primary)
[  22/1877] hphhlipa_c   → cpd1548

In [ ]:
# ─── Cell: compute ∆G′° for all reactions with optional balance check ─────────────────
import warnings
import pandas as pd
import csv
import cobra
from equilibrator_api import ComponentContribution, Q_
from sqlalchemy.orm.exc import MultipleResultsFound

# ——— CONFIG —————————————————————————————————————————————————————————————
COBRA_MODEL_FILE    = '/Users/benedikthaarscheidt/M.Sc./master_thesis/Models/generic_models/E_coli/iML1515.xml'
MAPPING_CSV         = 'curated_mapping.csv'     # semicolon-delimited: metabolite_id;equilibrator_id
PH                  = 7.0
IONIC_STRENGTH      = 0.25
OUTPUT_CSV          = 'reaction_dG0_prime.csv'
# Toggle whether to skip unbalanced reactions before computing ΔG'
IGNORE_UNBALANCED   = True    # set True to compute ΔG' even if reaction is unbalanced
# ————————————————————————————————————————————————————————————————————————

# suppress warnings about missing ChemAxon analyses
warnings.filterwarnings("ignore", message="Cannot calculate Legendre transform for Compound*")

# 1) load metabolite → equilibrator ID mapping
df = pd.read_csv(MAPPING_CSV, sep=';')
df.columns = df.columns.str.strip()
met_col, eq_col = df.columns[0], df.columns[1]
df[met_col] = df[met_col].str.strip()
df[eq_col]  = df[eq_col].astype(str).str.strip()
met2eq = dict(zip(df[met_col], df[eq_col]))

# 2) load COBRA model
model = cobra.io.read_sbml_model(COBRA_MODEL_FILE)

# 3) prepare Equilibrator
cc = ComponentContribution()
cc.p_h            = Q_(PH, "")
cc.ionic_strength = Q_(IONIC_STRENGTH, "M")

# 4) compute ∆G′° for each reaction & write to CSV
computed = skipped = errors = 0
with open(OUTPUT_CSV, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['reaction_id', 'equation', 'delta_G0_prime_kJ_per_mol', 'status'])

    for rxn in model.reactions:
        # map metabolites to equilibrator IDs
        stoich = {}
        for met, coeff in rxn.metabolites.items():
            eid = met2eq.get(met.id)
            if not eid:
                stoich = None
                break
            stoich[eid] = stoich.get(eid, 0) + coeff

        # skip if unmapped or trivial
        if not stoich or len(stoich) < 2:
            skipped += 1
            writer.writerow([rxn.id, rxn.reaction, '', 'skipped (unmapped)'])
            continue

        # build reaction formula string
        left, right = [], []
        for eid, coeff in stoich.items():
            if coeff < 0:
                term = f"{abs(int(coeff))} {eid}" if abs(coeff) != 1 else eid
                left.append(term)
            else:
                term = f"{int(coeff)} {eid}" if coeff != 1 else eid
                right.append(term)
        formula = " + ".join(left) + " = " + " + ".join(right)

        # parse reaction
        try:
            phased = cc.parse_reaction_formula(formula)
        except MultipleResultsFound:
            errors += 1
            writer.writerow([rxn.id, formula, '', 'error (ambiguous ID)'])
            continue
        except Exception as e:
            errors += 1
            writer.writerow([rxn.id, formula, '', f'error: {e}'])
            continue

        # optional balance check
        if not IGNORE_UNBALANCED:
            if not phased.is_balanced():
                skipped += 1
                writer.writerow([rxn.id, formula, '', 'skipped (unbalanced)'])
                continue

        # compute ΔG' and extract nominal kJ/mol
        try:
            dg = cc.standard_dg_prime(phased)
            uq = dg.to('kJ/mol').magnitude    # may be uncertainties object
            val = getattr(uq, 'nominal_value', uq)
            dg_kj = float(val)
            writer.writerow([rxn.id, formula, dg_kj, 'ok'])
            computed += 1
        except Exception as e:
            errors += 1
            writer.writerow([rxn.id, formula, '', f'error: {e}'])

print(f"Done: {computed} computed, {skipped} skipped, {errors} errors")
print(f"Results written to '{OUTPUT_CSV}'")


/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/unc

Done: 1479 computed, 1216 skipped, 17 errors
Results written to 'reaction_dG0_prime.csv'


/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/uncertainties/core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/equilibrator_cache/reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=74593, inchi_key=None): Compound(id=74593, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/equilibrator_cache/reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=74402, inchi_key=None): Compound(id=74402, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/equilibrator_cache/reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=1168, inchi_key=None): Compound(id=1168, inchi_key=None) has not yet been analyzed by ChemAxon.
  warnings.warn(
/opt/anaconda3/envs/GNN/lib/python3.12/site-packages/equilibrator_cache/reaction.py:606: UserWarning: Cannot calculate Legendre transform for Compound(id=174, inchi_key=None): Compound(id=174, in


🎉 Done – wrote 2712 rows to reaction_dG0.csv
